# Notebook de la dimensión Tiempo - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [2]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2. Extracción de datos - Extract

In [3]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [4]:
# Intervalo de cada minuto durante el día
rango_tiempo = pd.date_range(start="2024-01-01 00:00:00", end="2024-01-01 23:59:00", freq="min")

In [5]:
# Crear DataFrame
df_tiempo = pd.DataFrame()
df_tiempo["hora_completa"] = rango_tiempo.time
df_tiempo

,hora_completa
0,00:00:00
1,00:01:00
2,00:02:00
3,00:03:00
4,00:04:00
...,...
1435,23:55:00
1436,23:56:00
1437,23:57:00
1438,23:58:00


In [6]:
# Extraer partes individuales
df_tiempo["hora"] = rango_tiempo.hour
df_tiempo["minuto"] = rango_tiempo.minute
df_tiempo["segundo"] = 0  # Ya que usamos frecuencia de minuto

In [7]:
# Clasificar por momento del día
def clasificar_momento(hora):
    if 5 <= hora < 12:
        return "Mañana"
    elif 12 <= hora < 17:
        return "Tarde"
    elif 17 <= hora < 21:
        return "Noche"
    else:
        return "Madrugada"

df_tiempo["momento_dia"] = df_tiempo["hora"].apply(clasificar_momento)
df_tiempo

,hora_completa,hora,minuto,segundo,momento_dia
0,00:00:00,0,0,0,Madrugada
1,00:01:00,0,1,0,Madrugada
2,00:02:00,0,2,0,Madrugada
3,00:03:00,0,3,0,Madrugada
4,00:04:00,0,4,0,Madrugada
...,...,...,...,...,...
1435,23:55:00,23,55,0,Madrugada
1436,23:56:00,23,56,0,Madrugada
1437,23:57:00,23,57,0,Madrugada
1438,23:58:00,23,58,0,Madrugada


In [8]:
from datetime import datetime

# Agregar columna de fecha de carga
df_tiempo["fecha_carga"] = datetime.today().date()

In [9]:
# Reordenar columnas
df_tiempo = df_tiempo[["hora", "minuto", "segundo", "hora_completa", "momento_dia", "fecha_carga"]]
df_tiempo

,hora,minuto,segundo,hora_completa,momento_dia,fecha_carga
0,0,0,0,00:00:00,Madrugada,2025-07-01
1,0,1,0,00:01:00,Madrugada,2025-07-01
2,0,2,0,00:02:00,Madrugada,2025-07-01
3,0,3,0,00:03:00,Madrugada,2025-07-01
4,0,4,0,00:04:00,Madrugada,2025-07-01
...,...,...,...,...,...,...
1435,23,55,0,23:55:00,Madrugada,2025-07-01
1436,23,56,0,23:56:00,Madrugada,2025-07-01
1437,23,57,0,23:57:00,Madrugada,2025-07-01
1438,23,58,0,23:58:00,Madrugada,2025-07-01


In [10]:
df_tiempo.insert(0, "id_tiempo", df_tiempo.reset_index().index + 1)

In [11]:
# Reordenar columnas
df_tiempo = df_tiempo[["id_tiempo", "hora", "minuto", "segundo", "hora_completa", "momento_dia", "fecha_carga"]]

## 4. Guardado de la dimensión - Load

In [12]:
# Guardar como archivo CSV
df_tiempo.to_csv("dim_tiempo.csv", index=False)

In [13]:
# Cargar en la bodega de datos OLAP
df_tiempo.to_sql("dim_tiempo", engine_bodega, if_exists="replace", index=False)

440

In [14]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_tiempo.csv")
print(df_verificacion.head())

   id_tiempo  hora  minuto  segundo hora_completa momento_dia fecha_carga
0          1     0       0        0      00:00:00   Madrugada  2025-07-01
1          2     0       1        0      00:01:00   Madrugada  2025-07-01
2          3     0       2        0      00:02:00   Madrugada  2025-07-01
3          4     0       3        0      00:03:00   Madrugada  2025-07-01
4          5     0       4        0      00:04:00   Madrugada  2025-07-01
